In [28]:
pip install tflearn

  Created wheel for tflearn: filename=tflearn-0.3.2-py3-none-any.whl size=128212 sha256=5cd422298f78e9155f290ccb44bd6f0959a86dd9849ba7b5b08ef0399a793f24
  Stored in directory: c:\users\anves5\appdata\local\pip\cache\wheels\14\e9\35\ac682b1d18f932dc39cf86f25b22bb70df2e80e45851e4f9f3
Successfully built tflearn
Note: you may need to restart the kernel to use updated packages.


In [2]:
# We can't just send raw images throw our convolutional neural network. First we need all the images to be the same size, and we will also greyscale them.
import cv2          # working with, mainly resizing, images
import numpy as np  # dealing with arrays
import os           # dealing with directories
from random import shuffle      # mixing up our currently ordered data that might lead our network astay in training
from tqdm import tqdm           # a nice pretty percentage bar for tasks.

TRAIN_DIR = 'C:/Users/anves5/Documents/Kaggle_Data/dogs_vs_cats/train'
TEST_DIR = 'C:/Users/anves5/Documents/Kaggle_Data/dogs_vs_cats/test'
IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = 'dogsvscats-{}-{}-model'.format(LR, '2conv-basic') # just so we remember which saved model is which, sizes must match

In [3]:
# We convert the images and labels to array information that we can pass through the network.

# help function. Since the images are named dog.1 or cat.2 and so on, the names dog and cat can be split out, and then be converted to an array.
def label_img(img):
    word_label = img.split('.')[-3]
    # conversion to one-hot array [cat,dog]
    #                             [much cat, no dog]
    if word_label == 'cat' : return [1,0]
    #                             [no cat, very doggo]
    elif word_label == 'dog' : return [0,1]

# greyscaling and resizing the images and then putting the images into an array 
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img), np.array(label)])
    shuffle(training_data)
    np.save('training_data.npy', training_data)
    return training_data


def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('.')[0]
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img), img_num])
    shuffle(testing_data)
    np.save('testing_data.npy', testing_data)
    return testing_data


In [4]:
# If the data_set isn't already created
#train_data = create_train_data()

# If the data_set is already created:
train_data = np.load('training_data.npy', allow_pickle=True)

In [8]:
# -----Defining the neural network-----
# A 2 layered convolutional neural network, with a fully connected layer, and an output layer.

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

# We load the model if we have one saved. Instead of creating a new one every time.
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

# We split the testing and training data
train = train_data[:-500]
test = train_data[-500:]

# We create our data arrays
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

# Now we fit for 3 epochs
model.fit({'input': X}, {'targets': Y}, n_epoch=3, validation_set=({'input': test_x}, {'targets': test_y}), snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

---------------------------------
Run id: dogsvscats-0.001-2conv-basic-model
Log directory: log/
INFO:tensorflow:Summary name Accuracy/Adam_0 (raw) is illegal; using Accuracy/Adam_0__raw_ instead.
INFO:tensorflow:Summary name Accuracy_1/Adam_1 (raw) is illegal; using Accuracy_1/Adam_1__raw_ instead.
---------------------------------
Training samples: 49000
Validation samples: 1000
--


InvalidArgumentError: You must feed a value for placeholder tensor 'target/Y' with dtype float and shape [?,2]
	 [[node target/Y (defined at C:\Users\anves5\anaconda3\envs\RL-Learning-v3\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]

Original stack trace for 'target/Y':
  File "C:\Users\anves5\anaconda3\envs\RL-Learning-v3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\anves5\anaconda3\envs\RL-Learning-v3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\anves5\anaconda3\envs\RL-Learning-v3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\anves5\anaconda3\envs\RL-Learning-v3\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Users\anves5\anaconda3\envs\RL-Learning-v3\lib\site-packages\ipykernel\kernelapp.py", line 583, in start
    self.io_loop.start()
  File "C:\Users\anves